In [455]:
import numpy as np
import plotly.graph_objects as go
import os
import pickle 
import pandas as pd

filepath = 'C:/Users/Michael Forsthofer/OneDrive - University of Sussex/BadenLab/2p-imaging/multiplane/St53/'
pkl_files = [f for f in os.listdir(filepath) if f.endswith('.pkl')]

roi_data = pd.DataFrame(columns=['stimulus', 'ot_depth', 'plane_depth', 'amplitude', 'size', 'xy_centroid','experiment_ID', 'stage'])

y_res = 64



In [456]:
def pool_variable(input_data, stim):
    data_to_get = input_data[stim]
    if len(data_to_get)>0:
        flattened_data = np.hstack(data_to_get)
    else:
        flattened_data = np.array([])
    return flattened_data

#def measure_rois(inptud_data, stim):
    

In [457]:
ot_depth = []
amplitude = []
integral = []
latency = []
size = []
stimulus = []
plane_depth = []
experiment_ID = []
stage = []
x_centroid = []
y_centroid = []
cluster_ID = []

for pkl_file in pkl_files[0:1]:
    with open(filepath+pkl_file, 'rb') as f:
        extracted_data_loaded = pickle.load(f)
        
        stims = list(extracted_data_loaded['depths'].keys())
        cur_plane_depth = extracted_data_loaded['metadata']['depth']
        cur_stage = filepath.split('St')[-1].split('/')[0]
        print(pkl_file, extracted_data_loaded['metadata'])
        
        for cur_stim in stims:

            cur_ot_depth = pool_variable(extracted_data_loaded['depths'], cur_stim)
            ot_depth.extend(cur_ot_depth)
            amplitude.extend(pool_variable(extracted_data_loaded['amplitudes'], cur_stim))
            size.extend(pool_variable(extracted_data_loaded['sizes'], cur_stim))
            x_centroid.extend(pool_variable(extracted_data_loaded['x_centroid'], cur_stim))
            y_centroid.extend(pool_variable(extracted_data_loaded['y_centroid'], cur_stim))
            integral.extend(pool_variable(extracted_data_loaded['activity_area'], cur_stim))
            plane_depth.extend(pool_variable(extracted_data_loaded['plane_depth'], cur_stim))
            latency.extend(pool_variable(extracted_data_loaded['latency'], cur_stim))
            #cluster_ID.extend(pool_variable(extracted_data_loaded['cluster_ID'], cur_stim))
            stimulus.extend(np.repeat(np.array(cur_stim), len(cur_ot_depth)))
            experiment_ID.extend(np.repeat(np.array(pkl_file[11:26]), len(cur_ot_depth)))
            stage.extend(np.repeat(np.array(cur_stage), len(cur_ot_depth)))

2024-10-14_x241014_53_cOTm.h5_processed.pkl {'SD_threshold': 2.5, 'correlation_threshold': 0.62, 'min_traces_per_cluster': 7, 'cluster_threshold': 45, 'depth': 350, 'stage': 53, 'cluster_monotony_threshold': 0.7}


In [458]:
pooled_data = pd.DataFrame({'ot_depth': ot_depth, 
                     'amplitude': amplitude, 
                     'size': size,
                     'x_centroid': x_centroid,
                     'integral': integral,
                     'latency': latency,
                     'y_centroid': y_centroid,
                     'stimtype': stimulus,
                     'plane_depth': plane_depth,
                     'experiment_id': experiment_ID,
                     #'cluster_ID' : cluster_ID,
                     'stage': stage})

In [459]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Get the number of unique 'stimtype' values
num_stimtypes = pooled_data['stimtype'].nunique()

# Create a subplot with 1 row and 'num_stimtypes' columns
fig = make_subplots(rows=1, cols=num_stimtypes, shared_yaxes=True)

# Loop through each unique value of 'stimtype'
for i, stim in enumerate(pooled_data['stimtype'].unique(), start=1):
    filtered_data = pooled_data[(pooled_data['stimtype'] == stim)] #&(pooled_data['experiment_id'] == 'x241014_53_cOTs')
    fig.add_trace(go.Histogram(y=filtered_data['ot_depth'], name=stim,
                               nbinsy=30, opacity=0.75),
                  row=1, col=i)
fig.update_xaxes(range=[0, 6])
fig.update_yaxes(range=[0, 100])
# Update the layout
fig.update_layout(barmode='overlay', yaxis_title='y_centroid', xaxis_title='Count',
                  width=150 * num_stimtypes, height=300, margin=dict(l=20, r=20, t=20, b=20))

# Display the figure
fig.show()


In [460]:
import plotly.graph_objects as go

fig = go.Figure()

# Loop through each unique value of 'stimtype'
for stim in pooled_data['stimtype'].unique():
    filtered_data = pooled_data[pooled_data['stimtype'] == stim]
    fig.add_trace(go.Histogram(x=filtered_data['x_centroid'], name=stim, 
                               ybins = dict(  
            start=0,
            end=512,
            size=50)
        ))  # Set nbinsx to desired number of bins

# Update the layout
fig.update_layout(barmode='overlay', yaxis_title='x_centroid', xaxis_title='Count', width=600, height=300, margin=dict(l=20, r=20, t=20, b=20),)
fig.update_traces(opacity=0.75)
fig.update_xaxes(range=[0, 128])  # Disable gridlines for x-axis
fig.update_yaxes(range=[0,64])  # Disable gridlines for y-axis

fig.show()

fig2 = go.Figure()

# Loop through each unique value of 'stimtype'
for stim in pooled_data['stimtype'].unique():
    filtered_data = pooled_data[pooled_data['stimtype'] == stim]
    fig2.add_trace(go.Histogram(y=filtered_data['y_centroid']%y_res, name=stim, 
                               ybins = dict(  
            # start=0,
            # end=128,
            size=10)
        ))  # Set nbinsx to desired number of bins

# Update the layout
fig2.update_layout(barmode='overlay', yaxis_title='y_centroid', xaxis_title='Count', width=400, height=400, margin=dict(l=20, r=20, t=20, b=20),)
fig2.update_traces(opacity=0.75)
#fig.update_xaxes(range=[0, 100])  # Disable gridlines for x-axis
#fig.update_yaxes(range=[0,128])  # Disable gridlines for y-axis

fig2.show()

In [461]:
import plotly.express as px
fig = px.scatter(x=pooled_data['x_centroid'], y=pooled_data['y_centroid']%y_res, color=pooled_data['stimtype'], size=abs(pooled_data['amplitude']), opacity=pooled_data['size']/ pooled_data['size'].max())

fig.update_layout(
    plot_bgcolor='rgba(0,0,0,0)',  # Transparent background for the plot area
    #paper_bgcolor='rgba(0,0,0,0)',  # Transparent background for the overall figure
)

# Update axes to remove gridlines
fig.update_xaxes(showgrid=False)  # Disable gridlines for x-axis
fig.update_yaxes(showgrid=False)  # Disable gridlines for y-axis

In [462]:
import plotly.express as px
import plotly.subplots as sp

# Get unique stimtypes
unique_stimtypes = pooled_data['stimtype'].unique()

# Create a subplot with a number of rows equal to the number of unique stimtypes
num_stimtypes = len(unique_stimtypes)
fig = sp.make_subplots(rows=num_stimtypes, cols=1, shared_xaxes=True, subplot_titles=unique_stimtypes)

# Loop through each stimtype and generate a scatter plot for each
for i, stimtype in enumerate(unique_stimtypes):
    # Filter the DataFrame for the current stimtype
    filtered_data = pooled_data[pooled_data['stimtype'] == stimtype]
    
    # Create the scatter plot for this stimtype
    scatter = px.scatter(
        x=filtered_data['x_centroid'],
        y=filtered_data['y_centroid'] % y_res,
        color=filtered_data['plane_depth'],
        size=abs(filtered_data['amplitude']),
        opacity=filtered_data['size'] / filtered_data['size'].max()
    )

    # Add the scatter plot to the subplot
    for trace in scatter['data']:
        fig.add_trace(trace, row=i + 1, col=1)

# Update layout
fig.update_layout(coloraxis=dict(colorscale='Plasma', colorbar=dict(thickness=15, len=0.17, yanchor='bottom', y=-0.01)), 
                  height=300 * num_stimtypes, width=600, showlegend=False, 
                  margin=dict(l=20, r=20, t=20, b=20))

# Set axis ranges
fig.update_xaxes(range=[0, 128])
fig.update_yaxes(range=[0, 64])

# Show the figure
fig.show()

In [463]:
# Filter the DataFrame to include only rows where stimtype is 'loom'
filtered_data = pooled_data[(pooled_data['stimtype'] == 'loom')]

# Create the scatter plot using the filtered DataFrame
fig = px.scatter(
    x=filtered_data['x_centroid'],
    y=filtered_data['y_centroid']%y_res,
    color=filtered_data['plane_depth'],
    size=abs(filtered_data['amplitude']),
    opacity=filtered_data['size'] / filtered_data['size'].max()
)
fig.update_layout(width=600, height=300, margin=dict(l=20, r=20, t=20, b=20))
fig.update_xaxes(range=[0, 128])
fig.update_yaxes(range=[0, 64])
fig.show()

In [464]:
range = 913-240
pixelrange = 128
scaler = range/pixelrange

import plotly.express as px

# Filter the DataFrame to include only rows where stimtype is 'loom'
filtered_data = pooled_data[(pooled_data['stimtype'] == 'loom')]

# Create the 3D scatter plot using the filtered DataFrame
fig = px.scatter_3d(
    x=filtered_data['x_centroid']*scaler,
    y=(filtered_data['y_centroid'] % y_res) * scaler,
    z=-filtered_data['plane_depth'],  # Use plane_depth as the z-axis
    color=filtered_data['plane_depth'],
    size=abs(filtered_data['amplitude']),
    opacity=1
)

# Update layout with desired width, height, and margins
fig.update_layout(width=600, height=300, margin=dict(l=20, r=20, t=20, b=20))

# Set axis ranges
fig.update_xaxes(range=[0, 128])
fig.update_yaxes(range=[0, 64])

# Show the figure
fig.show()


In [465]:
import plotly.express as px

# Filter the DataFrame to include only the necessary rows
filtered_data = pooled_data[(pooled_data['stimtype'] == 'moving stripes')]  # You don't need to filter for 'loom' only anymore

# Calculate scaler
range_val = 913 - 240
pixelrange = 128
scaler = range_val / pixelrange

# Create the 3D scatter plot, using stimtype for color
fig = px.scatter_3d(
    x=filtered_data['x_centroid'] * scaler,
    y=(filtered_data['y_centroid'] % y_res) * scaler,
    z=-filtered_data['plane_depth'],  # Use plane_depth as the z-axis
    color=filtered_data['experiment_id'],  # Color based on stimtype
    size=abs(filtered_data['size']),
    opacity=0.6
)

# Update layout with desired width, height, and margins
fig.update_layout(width=800, height=600, margin=dict(l=20, r=20, t=20, b=20))

# Set axis ranges
fig.update_xaxes(range=[0, 128])
fig.update_yaxes(range=[0, 64])

# Show the figure
fig.show()

In [466]:
import plotly.graph_objects as go
import numpy as np

# Calculate scaler
range_val = 913 - 240
pixelrange = 128
scaler = range_val / pixelrange

# All data (background)
all_data = pooled_data

# Filtered data
filtered_data = pooled_data[pooled_data['stimtype'] == stimtype]

# Create the figure
fig = go.Figure()

# Add all data to the plot (background)
fig.add_trace(go.Scatter3d(
    x=all_data['x_centroid'] * scaler,
    y=(all_data['y_centroid'] % y_res) * scaler,
    z=-all_data['plane_depth'],
    mode='markers',
    marker=dict(
        size=abs(all_data['size']),
        color='lightgray',  # Light color for background
        opacity=0.5  # Low opacity to make it background-like
    ),
    name='All Data'
))

# Map experiment_id to a unique color
experiment_ids = filtered_data['experiment_id'].unique()
colors = px.colors.qualitative.D3  # You can choose another colorscale if you like
color_map = {experiment_id: colors[i % len(colors)] for i, experiment_id in enumerate(experiment_ids)}

# Apply the color map to the filtered data
filtered_data['color'] = filtered_data['experiment_id'].map(color_map)

# Add filtered data on top (foreground)
fig.add_trace(go.Scatter3d(
    x=filtered_data['x_centroid'] * scaler,
    y=(filtered_data['y_centroid'] % y_res) * scaler,
    z=-filtered_data['plane_depth'],
    mode='markers',
    marker=dict(
        size=abs(filtered_data['size']),
        color=filtered_data['color'],  # Apply color based on experiment_id
        opacity=1
    ),
    name='Filtered Data (Moving Stripes)'
))

# Update layout with desired width, height, and margins
fig.update_layout(
    width=800,
    height=600,
    margin=dict(l=20, r=20, t=20, b=20),
    #scene=dict(
        #xaxis=dict(range=[0, 128]),
        #yaxis=dict(range=[0, 64])
    #)
)

# Show the figure
fig.show()


C:\Users\Michael Forsthofer\AppData\Local\Temp\ipykernel_12128\1468738815.py:38: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [489]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# Calculate scaler
range_val = 913 - 240
pixelrange = 128
scaler = range_val / pixelrange

# All data (background)
all_data = pooled_data

# Get unique stimtypes
stimtypes = pooled_data['stimtype'].unique()

# Create a subplot figure
fig = make_subplots(
    rows=1, cols=len(stimtypes),  # One row and one column for each stimtype
    specs=[[{'type': 'scatter3d'}] * len(stimtypes)],  # Make each subplot a 3D scatter plot
    subplot_titles=stimtypes  # Titles for each subplot
)

# Define the camera view (adjust this as per your preferred xyz rotation)
camera = dict(
    eye=dict(x=0.95, y=1.25, z=0.5)  # This sets the viewing angle
)

# Add all data to each subplot as the background
for i, stimtype in enumerate(stimtypes):
    # Filter data by stimtype
    filtered_data = pooled_data[pooled_data['stimtype'] == stimtype]
    
    # Map experiment_id to a unique color
    experiment_ids = filtered_data['experiment_id'].unique()
    colors = ['maroon']  # You can choose another colorscale if you like
    color_map = {experiment_id: colors[j % len(colors)] for j, experiment_id in enumerate(experiment_ids)}

    # Apply the color map to the filtered data
    filtered_data['color'] = filtered_data['experiment_id'].map(color_map)

    # Add all data (background)
    fig.add_trace(go.Scatter3d(
        x=all_data['x_centroid'] * scaler,
        y=(all_data['y_centroid'] % y_res) * scaler,
        z=-all_data['plane_depth'],
        mode='markers',
        marker=dict(
            size=abs(all_data['amplitude'])*2,
            color='gray',  # Light color for background
            opacity=0.8  # Low opacity to make it background-like
        ),
        name=''
    ), row=1, col=i+1)

    # Add filtered data for the specific stimtype (foreground)
    fig.add_trace(go.Scatter3d(
        x=filtered_data['x_centroid'] * scaler,
        y=(filtered_data['y_centroid'] % y_res) * scaler,
        z=-filtered_data['plane_depth'],
        mode='markers',
        marker=dict(
            size=abs(filtered_data['amplitude'])*2,
            color=filtered_data['color'],  # Apply color based on experiment_id
            opacity=1
        ),
        name=stimtype
    ), row=1, col=i+1)
    fig.update_scenes(camera=camera, row=1, col=i+1)

# Update layout with desired width, height, and margins
fig.update_layout(
    height=400,
    width=1800,  # Adjust the width to fit all subplots
    margin=dict(l=20, r=20, t=40, b=20)
)

# Show the figure
fig.show()


C:\Users\Michael Forsthofer\AppData\Local\Temp\ipykernel_12128\3503393773.py:39: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Michael Forsthofer\AppData\Local\Temp\ipykernel_12128\3503393773.py:39: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Michael Forsthofer\AppData\Local\Temp\ipykernel_12128\3503393773.py:39: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docu

In [468]:
import plotly.express as px
import numpy as np
import pandas as pd

# Create empty lists to accumulate the data
x_vals = []
y_vals = []

# Loop to accumulate data
for id in np.unique(experiment_ID):
    for idx, stim in enumerate(np.unique(stimulus)):
        filtered_data = pooled_data[(pooled_data['stimtype'] == stim) & (pooled_data['experiment_id'] == id)]
        responsive_area = filtered_data['size'].sum()
        x_vals.append(stim)
        y_vals.append(responsive_area)

# Create a DataFrame for easier calculation of means
data = pd.DataFrame({'x': x_vals, 'y': y_vals})

# Calculate the mean for each unique x value
means = data.groupby('x')['y'].mean().reset_index()

# Create the scatter plot after the loop
fig = px.scatter(x=x_vals, y=y_vals, labels={'y': 'Responsive Area [px]'})

# Update the layout and traces for the main scatter plot
fig.update_layout(width=300, height=400, margin=dict(l=20, r=20, t=20, b=20))
fig.update_traces(marker=dict(size=9, color='grey'))
fig.update_xaxes(range=[-1, 4])
#fig.update_yaxes(range=[-100, 4000])

# Style the plot
fig.update_layout(
    plot_bgcolor='white',
    xaxis=dict(
        showline=True, linewidth=2, linecolor='black', mirror=False,
        tickvals=[0, 1, 2, 3],
        ticks='outside', ticklen=10, tickwidth=2, tickcolor='black'
    ),
    yaxis=dict(
        showline=True, linewidth=2, linecolor='black', mirror=False
    )
)

# Overlay black dots for the mean values
fig.add_scatter(
    x=means['x'], y=means['y'], 
    mode='markers', 
    marker=dict(size=12, color='crimson', symbol='circle'), 
    name='Mean'
)

# Show the plot
fig.show()


In [469]:
px.scatter(pooled_data, x='ot_depth', y='integral', color=pooled_data['stimtype'], opacity=0.9)

In [470]:
px.histogram(pooled_data, x='latency', color=pooled_data['stimtype'], opacity=0.7, nbins=40)
